In [1]:
import math
import numpy as np
from scipy import signal
from scipy.signal import butter, lfilter
import IPython.display as ipd
import matplotlib.pyplot as plt
from config import SAMPLE_RATE, nsamples, sample_space, angular_freq, amp

In [2]:
def asdf_envelope(
    wave,
    attack_duration=0, 
    decay_duration=0, 
    sustain_duration=0,
    release_duration=0,
    sustain_level=1,
    linear=True
):
    envelope = np.ones(len(sample_space))

    # Apply attack envelope
    attack_samples = int(attack_duration * SAMPLE_RATE)
    envelope[:attack_samples] = np.linspace(0, 1, attack_samples)

    # Apply sustain envelope
    sustain_samples = int(sustain_duration * SAMPLE_RATE)
    start_sustain = attack_samples
    envelope[start_sustain:start_sustain+sustain_samples] = sustain_level

    # Apply decay envelope
    decay_samples = int(decay_duration * SAMPLE_RATE)
    start_decay = start_sustain + sustain_samples
    envelope[start_decay:start_decay+decay_samples] = np.linspace(sustain_level, 0, decay_samples)

    # Apply fade-out envelope
    release_samples = int(release_duration * SAMPLE_RATE)
    start_release = len(sample_space) - release_samples
    envelope[start_release:] = np.linspace(0, 0, release_samples)
    
    if linear:
        envelope = envelope * np.linspace(0, 1, len(sample_space))
    else:
        envelope = envelope * np.exp(sample_space)
    
    # Generate the modulated sine wave
    wave_asdf = amp * envelope * wave

    return wave_asdf

def resonant_filter(input_wave, resonance_q, sample_rate=SAMPLE_RATE):
    # Calculate the center frequency of the filter
    center_freq = 440.0  # Replace with your desired center frequency in Hz
    w0 = 2 * np.pi * center_freq / sample_rate

    # Calculate the bandwidth from the resonance Q
    bandwidth = center_freq / resonance_q

    # Design a bandpass filter
    b, a = butter(
        N=2,
        Wn=[(center_freq - bandwidth/2) / sample_rate, (center_freq + bandwidth/2) / sample_rate],
        btype='band'
    )

    # Apply the filter to the input wave
    filtered_wave = lfilter(b, a, input_wave)

    return filtered_wave

In [7]:
# Sawtooth wave
frequency = 500
a0 = signal.sawtooth(angular_freq(frequency))*amp

a2 = asdf_envelope(
    a0,
    attack_duration=0.01, 
    decay_duration=0, 
    sustain_duration=1,
    release_duration=0.01,
)

a3 = resonant_filter(
    a2,
    resonance_q=30
)

In [8]:
ipd.Audio(a3, rate=SAMPLE_RATE)